In [1]:
#from google.colab import drive
#drive.mount('/content/drive')
import sys
#PATH = '/content/drive/My Drive/PPM_Stability/'
#PATH = "C:/Users/velmurug/Documents/Stability Experiments/benchmark_interpretability/PPM_Stability/"
#PATH = "C:/Users/Mythreyi/Documents/GitHub/Stability-Experiments/benchmark_interpretability/PPM_Stability/"
PATH = "C:/Users/mythr/Documents/GitHub/Stability-Experiments/benchmark_interpretability/PPM_Stability/"
sys.path.append(PATH)

In [2]:
#!pip install lime
#!pip install shap
#!pip install pandas==0.19.2
!pip install xgboost==1.0.0

In [3]:
import EncoderFactory
from DatasetManager import DatasetManager
import BucketFactory
import stability as st #Nogueira, Sechidis, Brown.

import pandas as pd
import numpy as np
from scipy import stats

from sklearn.metrics import roc_auc_score
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import StandardScaler

import time
import os
import sys
from sys import argv
import pickle
from collections import defaultdict
import random
import joblib

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import lime
import lime.lime_tabular
from lime import submodular_pick;

import shap

import warnings
warnings.filterwarnings('ignore')

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

def imp_df(column_names, importances):
        df = pd.DataFrame({'feature': column_names,
                       'feature_importance': importances}) \
           .sort_values('feature_importance', ascending = False) \
           .reset_index(drop = True)
        return df

# plotting a feature importance dataframe (horizontal barchart)
def var_imp_plot(imp_df, title, num_feat):
        imp_df.columns = ['feature', 'feature_importance']
        b= sns.barplot(x = 'feature_importance', y = 'feature', data = imp_df.head(num_feat), orient = 'h', palette="Blues_r")

In [5]:
def generate_global_explanations(train_X,train_Y, cls, feature_combiner):
    
    print("The number of testing instances is ",len(train_Y))
    print("The total number of columns is",train_X.shape[1]);
    print("The total accuracy is ",cls.score(train_X,train_Y));
       
    sns.set(rc={'figure.figsize':(10,10), "font.size":18,"axes.titlesize":18,"axes.labelsize":18})
    sns.set
    feat_names = feature_combiner.get_feature_names()
    base_imp = imp_df(feat_names, cls.feature_importances_)
    base_imp.head(15)
    var_imp_plot(base_imp, 'Feature importance using XGBoost', 15)
    return base_imp

In [6]:
from lime import submodular_pick
def generate_lime_explanations(explainer,test_xi, cls,test_y, submod=False, test_all_data=None, max_feat = 10):
    
    #print("Actual value ", test_y)
    exp = explainer.explain_instance(test_xi, 
                                 cls.predict_proba, num_features=max_feat, labels=[0,1])
    
    return exp
        
    if submod==True:
        sp_obj=submodular_pick.SubmodularPick(explainer, test_all_data, cls.predict_proba, 
                                      sample_size=20, num_features=num_features,num_exps_desired=4)
        [exp.as_pyplot_figure(label=exp.available_labels()[0]) for exp in sp_obj.sp_explanations];

In [7]:
def create_samples(shap_explainer, iterations, row, features, top = None):
    length = len(features)
    
    exp = []
    rel_exp = []
    
    for j in range(iterations):
        
        #if shap_type == "kernel":
        #    shap_explainer = shap.KernelExplainer(cls.predict, trainingsample)
        #elif shap_type == "tree":
        #    shap_explainer = shap.TreeExplainer(cls)
        #elif shap_type == "deep":
        #    shap_explainer = shap.DeepExplainer(cls, background)
        
        #print(X_test_frame.loc[row])
        shap_values = shap_explainer.shap_values(row)
        #print(shap_values)

        importances = []
        
        if type(shap_explainer) == shap.explainers.kernel.KernelExplainer:
            for i in range(length):
                feat = features[i]
                shap_val = shap_values[0][i]
                abs_val = abs(shap_values[0][i])
                entry = (feat, shap_val, abs_val)
                importances.append(entry)
                
        elif type(shap_explainer) == shap.explainers.tree.TreeExplainer:
            for i in range(length):
                feat = features[i]
                shap_val = shap_values[0][i]
                abs_val = abs(shap_values[0][i])
                entry = (feat, shap_val, abs_val)
                importances.append(entry)
        
        elif type(shap_explainer) == shap.explainers.deep.DeepExplainer:
            for i in range(length):
                feat = features[i]
                shap_val = shap_values[0][0][i]
                abs_val = abs(shap_values[0][0][i])
                entry = (feat, shap_val, abs_val)
                importances.append(entry)
        
        importances.sort(key=lambda tup: tup[2], reverse = True)
        
        exp.append(importances)

        rel_feat = []

        if top != None:
            for i in range(top):
                feat = importances[i]
                if feat[2] > 0:
                    rel_feat.append(feat)

            rel_exp.append(rel_feat)
        else:
            rel_exp = exp
        
    return exp, rel_exp

In [8]:
def dispersal(weights, features):
    feat_len = len(features)
    #print(feat_len)
    weights_by_feat = []
       
    for i in list(range(feat_len)):
        feat_weight = []
        for iteration in weights:
            feat_weight.append(iteration[i])
        weights_by_feat.append(feat_weight)
        
    #for iteration in weights:
     #   for val in iteration:
      #      idx = iteration.index(val)
       #     print(idx)
        #    weights_by_feat[idx].append(val)
    
    dispersal = []
    dispersal_no_outlier = []
    
    for each in weights_by_feat:
        #print("Feature", weights_by_feat.index(each)+1)
        mean = np.mean(each)
        std_dev = np.std(each)
        var = std_dev**2
        
        if mean == 0:
            dispersal.append(0)
            dispersal_no_outlier.append(0)
        #print(each)
        else:
            #dispersal with outliers
            rel_var = var/abs(mean)
            dispersal.append(rel_var)
            
            #dispersal without outliers - remove anything with a z-score higher
            #than 3 (more than 3 standard deviations away from the mean)
            rem_outlier = []
            z_scores = stats.zscore(each)
            #print(z_scores)
            #print("New list:")
            for i in range(len(z_scores)):
                #print(each[i],":",z_scores[i])
                if -3 < z_scores[i] < 3:
                    rem_outlier.append(each[i])
                #print(rem_outlier)
            if rem_outlier != []:
                new_mean = np.mean(rem_outlier)
                if new_mean == 0:
                    dispersal_no_outlier.append(0)
                else:
                    new_std = np.std(rem_outlier)
                    new_var = new_std**2
                    new_rel_var = new_var/abs(new_mean)
                    dispersal_no_outlier.append(new_rel_var)
            else:
                dispersal_no_outlier.append(rel_var)
    #print(dispersal_no_outlier)
    return dispersal, dispersal_no_outlier

In [9]:
dataset_ref = "bpic2012"
params_dir = PATH + "params"
results_dir = "results"
bucket_method = "prefix"
cls_encoding = "index"
cls_method = "xgboost"

gap = 1
n_iter = 1

method_name = "%s_%s"%(bucket_method, cls_encoding)

generate_samples = False
generate_lime = True
generate_kernel_shap = False
generate_model_shap = True

sample_size = 2
exp_iter = 10
max_feat = 10

dataset_ref_to_datasets = {
    #"bpic2011": ["bpic2011_f%s"%formula for formula in range(1,5)],
    "bpic2015": ["bpic2015_%s_f2"%(municipality) for municipality in range(5,6)],
    "bpic2017" : ["bpic2017_accepted"],
    "bpic2012" : ["bpic2012_accepted"]
    #"insurance": ["insurance_activity", "insurance_followup"],
    #"sepsis_cases": ["sepsis_cases_1", "sepsis_cases_2", "sepsis_cases_4"]
}

datasets = [dataset_ref] if dataset_ref not in dataset_ref_to_datasets else dataset_ref_to_datasets[dataset_ref]

datasets

['bpic2012_accepted']

In [10]:
for dataset_name in datasets:

    dataset_manager = DatasetManager(dataset_name)

    for ii in range(n_iter):
        num_buckets = range(len([name for name in os.listdir(os.path.join(PATH,'%s/%s_%s/models'% (dataset_ref, cls_method, method_name)))]))

        for bucket in list(num_buckets):
            bucketID = bucket+1
            print ('Bucket', bucketID)

            #import everything needed to sort and predict
            pipeline_path = os.path.join(PATH, "%s/%s_%s/pipelines/pipeline_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
            feat_comb_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/feature_combiner_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
            bucketer_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/bucketer_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
            cls_path = os.path.join(PATH, "%s/%s_%s/models/cls_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))

            predictor = joblib.load(pipeline_path)
            cls = joblib.load(cls_path)
            feature_combiner = joblib.load(feat_comb_path)
            bucketer = joblib.load(bucketer_path)

            #import data for bucket
            X_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))
            Y_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucketID))
            X_test_path = os.path.join(PATH, "%s/%s_%s/test_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))
            Y_test_path = os.path.join(PATH, "%s/%s_%s/test_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucketID))

            with open (X_train_path, 'rb') as f:
                dt_train_bucket = pickle.load(f)
            with open (Y_train_path, 'rb') as f:
                train_y = pickle.load(f)
            with open (X_test_path, 'rb') as f:
                dt_test_bucket = pickle.load(f)
            with open (Y_test_path, 'rb') as f:
                test_y = pickle.load(f)

            #import previously defined samples
            tn_path = os.path.join(PATH, "%s/%s_%s/samples/true_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
            tp_path = os.path.join(PATH, "%s/%s_%s/samples/true_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
            fn_path = os.path.join(PATH, "%s/%s_%s/samples/false_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
            fp_path = os.path.join(PATH, "%s/%s_%s/samples/false_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))

            sample_instances = []

            with open (tn_path, 'rb') as f:
                tn_list = pickle.load(f)
            with open (tp_path, 'rb') as f:
                tp_list = pickle.load(f)
            #with open (fn_path, 'rb') as f:
            #    fn_list = pickle.load(f)
            #with open (fp_path, 'rb') as f:
            #    fp_list = pickle.load(f)

            #save results to a list
            sample_instances.append(tn_list)
            sample_instances.append(tp_list)
            #sample_instances.append(fn_list)
            #sample_instances.append(fp_list)

            #print(cls)

Bucket 1
XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7047348347942339, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints=None,
              learning_rate=0.08635792869395631, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=500, n_jobs=0, num_parallel_tree=1, random_state=22,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=22,
              subsample=0.5627056016547434, tree_method=None,
              validate_parameters=False, verbosity=None)
Bucket 2
XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7047348347942339, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints=None,
              learning_rate=0.08635792869395631, max_delta_step=0, max_depth=9,
              min_child_weight=

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7047348347942339, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints=None,
              learning_rate=0.08635792869395631, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=500, n_jobs=0, num_parallel_tree=1, random_state=22,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=22,
              subsample=0.5627056016547434, tree_method=None,
              validate_parameters=False, verbosity=None)
Bucket 15
XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7047348347942339, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints=None,
              learning_rate=0.08635792869395631, max_delta_step=0, max_depth=9,
              min_child_weight=1, missi

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7047348347942339, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints=None,
              learning_rate=0.08635792869395631, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=500, n_jobs=0, num_parallel_tree=1, random_state=22,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=22,
              subsample=0.5627056016547434, tree_method=None,
              validate_parameters=False, verbosity=None)
Bucket 28
XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7047348347942339, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints=None,
              learning_rate=0.08635792869395631, max_delta_step=0, max_depth=9,
              min_child_weight=1, missi

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7047348347942339, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints=None,
              learning_rate=0.08635792869395631, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=500, n_jobs=0, num_parallel_tree=1, random_state=22,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=22,
              subsample=0.5627056016547434, tree_method=None,
              validate_parameters=False, verbosity=None)


In [13]:
input_ = sample_instances[0][0]['input']
input_

AMOUNT_REQ    Case ID    label                         Activity  \
161331     35000.0  206357_40  regular             A_SUBMITTED-COMPLETE   
161332     35000.0  206357_40  regular       A_PARTLYSUBMITTED-COMPLETE   
161333     35000.0  206357_40  regular           A_PREACCEPTED-COMPLETE   
161334     35000.0  206357_40  regular  W_Completeren aanvraag-SCHEDULE   
161335     35000.0  206357_40  regular     W_Completeren aanvraag-START   
161336     35000.0  206357_40  regular              A_ACCEPTED-COMPLETE   
161338     35000.0  206357_40  regular             A_FINALIZED-COMPLETE   
161337     35000.0  206357_40  regular              O_SELECTED-COMPLETE   
161339     35000.0  206357_40  regular               O_CREATED-COMPLETE   
161340     35000.0  206357_40  regular                  O_SENT-COMPLETE   
161341     35000.0  206357_40  regular     W_Nabellen offertes-SCHEDULE   
161342     35000.0  206357_40  regular  W_Completeren aanvraag-COMPLETE   
161343     35000.0  206357_40  regular        W_Nabellen offertes-START   
161344     35000.0  206357_40  regular             O_CANCELLED-COMPLETE   
161345     35000.0  206357_40  regular              O_SELECTED-COMPLETE   
161346     35000.0  206357_40  regular               O_CREATED-COMPLETE   
161347     35000.0  206357_40  regular                  O_SENT-COMPLETE   
161348     35000.0  206357_40  regular     W_Nabellen offertes-SCHEDULE   
161349     35000.0  206357_40  regular     W_Nabellen offertes-COMPLETE   
161350     35000.0  206357_40  regular        W_Nabellen offertes-START   
161352     35000.0  206357_40  regular             O_CANCELLED-COMPLETE   
161351     35000.0  206357_40  regular              O_SELECTED-COMPLETE   
161353     35000.0  206357_40  regular               O_CREATED-COMPLETE   
161354     35000.0  206357_40  regular                  O_SENT-COMPLETE   
161355     35000.0  206357_40  regular     W_Nabellen offertes-SCHEDULE   
161356     35000.0  206357_40  regular     W_Nabellen offertes-COMPLETE   
161357     35000.0  206357_40  regular        W_Nabellen offertes-START   
161358     35000.0  206357_40  regular             O_CANCELLED-COMPLETE   
161359     35000.0  206357_40  regular              O_SELECTED-COMPLETE   
161360     35000.0  206357_40  regular               O_CREATED-COMPLETE   
161361     35000.0  206357_40  regular                  O_SENT-COMPLETE   
161362     35000.0  206357_40  regular     W_Nabellen offertes-SCHEDULE   
161363     35000.0  206357_40  regular     W_Nabellen offertes-COMPLETE   
161364     35000.0  206357_40  regular        W_Nabellen offertes-START   
161365     35000.0  206357_40  regular     W_Nabellen offertes-COMPLETE   
161366     35000.0  206357_40  regular        W_Nabellen offertes-START   
161367     35000.0  206357_40  regular     W_Nabellen offertes-COMPLETE   
161368     35000.0  206357_40  regular        W_Nabellen offertes-START   
161369     35000.0  206357_40  regular     W_Nabellen offertes-COMPLETE   
161370     35000.0  206357_40  regular        W_Nabellen offertes-START   

       Resource lifecycle:transition  timesincemidnight  timesincelastevent  \
161331    112.0             COMPLETE             1254.0            0.000000   
161332    112.0             COMPLETE             1254.0            0.010483   
161333    112.0             COMPLETE             1255.0            0.654500   
161334    112.0             SCHEDULE             1255.0            0.016450   
161335    112.0                START             1262.0            6.907683   
161336  11202.0             COMPLETE             1272.0           10.353267   
161338  11202.0             COMPLETE             1276.0            3.965933   
161337  11202.0             COMPLETE             1276.0            0.000000   
161339  11202.0             COMPLETE             1276.0            0.018667   
161340  11202.0             COMPLETE             1276.0            0.000500   
161341  11202.0             SCHEDULE             1276.0        

In [15]:
test_x = feature_combiner.fit_transform(input_)
test_x

array([[3.5e+04, 2.0e+01, 4.0e+00, ..., 0.0e+00, 0.0e+00, 0.0e+00]])

In [18]:
col = feature_combiner.get_feature_names()
transformed = pd.DataFrame(test_x, columns = col)
transformed

static__AMOUNT_REQ  index__hour_0  index__weekday_0  index__month_0  \
0             35000.0           20.0               4.0             2.0   

   index__timesincemidnight_0  index__timesincelastevent_0  \
0                      1254.0                          0.0   

   index__timesincecasestart_0  index__event_nr_0  index__open_cases_0  \
0                          0.0                1.0                848.0   

   index__hour_1  ...  index__Resource_39_11181.0  index__Resource_39_11189.0  \
0           20.0  ...                         0.0                         0.0   

   index__Resource_39_112.0  index__Resource_39_11200.0  \
0                       0.0                         0.0   

   index__Resource_39_11201.0  index__Resource_39_11202.0  \
0                         0.0                         0.0   

   index__Resource_39_11203.0  index__Resource_39_11259.0  \
0                         0.0                         0.0   

   index__Resource_39_11289.0  index__Resource_39_other  
0                         0.0                       0.0  

[1 rows x 2829 columns]